# Q4 Shoulders of Giants (15 points)
As we have already seen, deep networks can sometimes be hard to optimize. Often times they heavily overfit on small training sets. Many approaches have been proposed to counter this, eg, [Krahenbuhl et al. (ICLR’16)](http://arxiv.org/pdf/1511.06856.pdf), self-supervised learning, etc. However, the most effective approach remains pre-training the network on large, well-labeled supervised datasets such as ImageNet. 

While training on the full ImageNet data is beyond the scope of this assignment, people have already trained many popular/standard models and released them online. In this task, we will initialize a ResNet-18 model with pre-trained ImageNet weights (from `torchvision`), and finetune the network for PASCAL classification.

## 4.1 Load Pre-trained Model (7 pts)\
Load the pre-trained weights up to the second last layer, and initialize last weights and biases from scratch.

The model loading mechanism is based on names of the weights. It is easy to load pretrained models from `torchvision.models`, even when your model uses different names for weights. Please briefly explain how to load the weights correctly if the names do not match ([hint](https://discuss.pytorch.org/t/loading-weights-from-pretrained-model-with-different-module-names/11841)).

**YOUR ANSWER HERE**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import matplotlib.pyplot as plt
%matplotlib inline

import trainer
from utils import ARGS
from simple_cnn import SimpleCNN
from voc_dataset import VOCDataset


# Pre-trained weights up to second-to-last layer
# final layers should be initialized from scratcH!
class PretrainedResNet(nn.Module):
    def __init__(self):
        super().__init__()
        ResNet = models.resnet18(pretrained=True)
        self.backbone = nn.Sequential(*list(ResNet.children())[:-1])
        for param in self.backbone:
            param.requires_grad = False
        self.classifier = nn.Linear(512,20,bias=True)
    
    def forward(self, x):
        features = self.backbone(x)
        features = features.view(-1, 512)
        out = self.classifier(features)
        return out
        

Use similar hyperparameter setup as in the scratch case. Show the learning curves (training loss, testing MAP) for 10 epochs. Please evaluate your model to calculate the MAP on the testing dataset every 100 iterations.

**REMEMBER TO SAVE MODEL AT END OF TRAINING**

In [2]:
args = ARGS(batch_size=32, test_batch_size=32, epochs=10, val_every=100, lr=1e-4, size=227, save_freq=1)
model = PretrainedResNet()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=args.gamma)
test_ap, test_map = trainer.train(args, model, optimizer, scheduler, model_name='runs/q4/model')
print('test map:', test_map)

Train Epoch: 0 [0 (0%)]	Loss: 0.738947
Train Epoch: 0 [100 (64%)]	Loss: 0.158812
Train Epoch: 1 [200 (27%)]	Loss: 0.077691
Train Epoch: 1 [300 (91%)]	Loss: 0.099643
Train Epoch: 2 [400 (55%)]	Loss: 0.080847
Train Epoch: 3 [500 (18%)]	Loss: 0.040495
Train Epoch: 3 [600 (82%)]	Loss: 0.040389
Train Epoch: 4 [700 (46%)]	Loss: 0.038934
Train Epoch: 5 [800 (10%)]	Loss: 0.021909
Train Epoch: 5 [900 (73%)]	Loss: 0.016995
Train Epoch: 6 [1000 (37%)]	Loss: 0.014961
Train Epoch: 7 [1100 (1%)]	Loss: 0.013639
Train Epoch: 7 [1200 (64%)]	Loss: 0.015578
Train Epoch: 8 [1300 (28%)]	Loss: 0.008688
Train Epoch: 8 [1400 (92%)]	Loss: 0.011300
Train Epoch: 9 [1500 (55%)]	Loss: 0.007435
test map: 0.817214616125489


**YOUR TB SCREENSHOTS HERE**

![title](imgs/q4_tb_map.png)
![title](imgs/q4_tb_loss.png)